In [1]:
from data.update.indian_equity import fill_gap
from data.fetch.indian_equity import fetch_symbol_list_indian_equity
from utils.db.fetch import fetch_entries
from executor.indian_equity_pipeline import run_pipeline
import pandas as pd
import schedule
import time
from executor.executor import execute_trades_telegram, execute_trades_zerodha, is_market_open, get_balance
from utils.notifier.telegram import send_telegram_message
from utils.data.dataframe import get_top_symbols_by_average_volume
import pytz
from datetime import datetime, timedelta
import os

/usr/local/lib/python3.11/site-packages/dash_bootstrap_components/_table.py:5: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


In [2]:
import sys
import os
# Add the parent directory to the Python path

# Change the current working directory to /app
os.chdir('/app')
from dotenv import load_dotenv
load_dotenv(dotenv_path='config/.env')
database_path = os.getenv('DATABASE_PATH')
from utils.db.fetch import fetch_entries
print(f'DATABASE_PATH: {database_path}')

DATABASE_PATH: database/db/ohlcv_data.db


In [15]:
import json
dict_string = os.getenv("TELEGRAM_BOT_CHANNELS")
my_dict = json.loads(dict_string)

In [23]:
send_telegram_message(message=stringied, chat_id=my_dict['4h_altbtc_momentum'])

In [4]:
send_telegram_message(message='ping test', chat_id=my_dict['debug_logs'])

Line by Line execution of fill gaps

In [3]:
from utils.db.fetch import fetch_latest_date
latest_date = fetch_latest_date(market_name='crypto_binance', timeframe='4h', storage_system='finstore', pair='BTC')
latest_date

Len items in 'database/finstore/market_name=crypto_binance/timeframe=4h': 476


Timestamp('2023-12-29 00:00:00')

In [4]:
from data.gather.crypto_binance import gather_ohlcv_binance
symbols, data = gather_ohlcv_binance(timeframe='4h', start_date=latest_date, type='spot', suffix='BTC')

100%|██████████| 476/476 [24:44<00:00,  3.12s/it]


In [6]:
data['1INCH/BTC']

,timestamp,open,high,low,close,volume
0,2023-12-29 00:00:00,0.000010,0.000010,0.000010,0.000010,98695.8
1,2023-12-29 04:00:00,0.000010,0.000010,0.000010,0.000010,93306.7
2,2023-12-29 08:00:00,0.000010,0.000011,0.000010,0.000010,55593.5
3,2023-12-29 12:00:00,0.000010,0.000010,0.000010,0.000010,67004.2
4,2023-12-29 16:00:00,0.000010,0.000010,0.000010,0.000010,49743.0
...,...,...,...,...,...,...
2101,2024-12-13 04:00:00,0.000005,0.000005,0.000005,0.000005,83178.5
2102,2024-12-13 08:00:00,0.000005,0.000005,0.000005,0.000005,29566.9
2103,2024-12-13 12:00:00,0.000005,0.000005,0.000005,0.000005,63216.3
2104,2024-12-13 16:00:00,0.000005,0.000005,0.000005,0.000005,96718.8


In [7]:

from data.store.crypto_binance import store_crypto_binance_gaps
store_crypto_binance_gaps(symbols, data, timeframe='4h', pair='BTC')


Len items in 'database/finstore/market_name=crypto_binance/timeframe=4h': 474


Writing symbols: 100%|██████████| 476/476 [00:03<00:00, 120.60it/s]


In [ ]:
ohlcv_data = fetch_entries(market_name='crypto_binance', timeframe='4h', all_entries=True, pair='BTC')
ohlcv_data['1INCH/BTC']

In [ ]:
from finstore.finstore import Finstore
finstore = Finstore(market_name='crypto_binance', timeframe='4h', enable_append=True, pair='BTC')
symbol_list = finstore.read.get_symbol_list()
symbol_list

In [ ]:
from data.update.crypto_binance import update_calculated_indicators
update_calculated_indicators(market_name='crypto_binance', symbol_list=symbol_list, timeframe='4h', all_entries=False, pair='BTC')